```
This notebook sets up and runs a set of benchmarks to compare
different numerical discretizations of the SWEs

Copyright (C) 2016  SINTEF ICT

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.
```

In [ ]:
#Lets have matplotlib "inline"
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

#Import packages we need
import numpy as np
from matplotlib import animation, rc
from matplotlib import pyplot as plt

import os
import pyopencl
import datetime
import sys

#Finally, import our simulator
from SWESimulators import FBL, CTCS, DataOutput

#Set large figure sizes
rc('figure', figsize=(16.0, 12.0))
rc('animation', html='html5')

#Import our simulator
from SWESimulators import FBL, CTCS, LxF, KP07, CDKLM16, DataOutput, PlotHelper, Common
#Import initial condition and bathymetry generating functions:
from SWESimulators.BathymetryAndICs import *

In [ ]:
#Make sure we get compiler output from OpenCL
os.environ["PYOPENCL_COMPILER_OUTPUT"] = "1"

#Set which CL device to use, and disable kernel caching
if (str.lower(sys.platform).startswith("linux")):
    os.environ["PYOPENCL_CTX"] = "0"
else:
    os.environ["PYOPENCL_CTX"] = "1"
os.environ["CUDA_CACHE_DISABLE"] = "1"
os.environ["PYOPENCL_COMPILER_OUTPUT"] = "1"
os.environ["PYOPENCL_NO_CACHE"] = "1"

#Create OpenCL context
cl_ctx = pyopencl.create_some_context()
print "Using ", cl_ctx.devices[0].name

In [ ]:
#Create output directory for images
imgdir='images_' + datetime.datetime.now().strftime("%Y_%m_%d-%H_%M_%S")
os.makedirs(imgdir)
print "Saving images to " + imgdir

In [ ]:
# Forward backward linear
nx = 100
ny = 200

dx = 200.0
dy = 200.0

dt = 1
g = 9.81
f = 0.0
r = 0.0

#wind = Common.WindStressParams(type=0, tau0=3.0, rho=1025, alpha=1.0/(100*dx))
wind = Common.WindStressParams(type=99)


bcSettings = 1
if (bcSettings == 1):
    boundaryConditions = Common.BoundaryConditions()
    ghosts = [0,0,0,0] # north, east, south, west
    validDomain = [None, None, 0, 0]
elif (bcSettings == 2):
    boundaryConditions = Common.BoundaryConditions(2,2,2,2)
    ghosts = [1,1,0,0] # Both periodic
    validDomain = [-1, -1, 0, 0]
elif bcSettings == 3:
    boundaryConditions = Common.BoundaryConditions(2,1,2,1)
    ghosts = [1,0,0,0] # periodic north-south
    validDomain = [-1, None, 0, 0]
else:
    boundaryConditions = Common.BoundaryConditions(1,2,1,2)
    ghosts = [0,1,0,0] # periodic east-west
    validDomain = [None, -1, 0, 0]


h0 = np.ones((ny+ghosts[0], nx+ghosts[1]), dtype=np.float32) * 60;
#addTopographyBump(h0, nx, ny, dx, dy, ghosts, 40)

eta0 = np.zeros((ny+ghosts[0], nx+ghosts[1]), dtype=np.float32);
u0 = np.zeros((ny+ghosts[0], nx+1), dtype=np.float32);
v0 = np.zeros((ny+1, nx+ghosts[1]), dtype=np.float32);

#Create bump in to lower left of domain for testing
addCentralBump(eta0, nx, ny, dx, dy, ghosts)


#Initialize simulator
reload(FBL)
sim = FBL.FBL(cl_ctx, \
              h0, eta0, u0, v0, \
              nx, ny, \
              dx, dy, dt, \
              g, f, r, \
              wind_stress=wind, \
              boundary_conditions=boundaryConditions)


#Calculate radius from center of bump for plotting
x_center = dx*nx*0.3
y_center = dy*ny*0.2
y_coords, x_coords = np.mgrid[0:ny*dy:dy, 0:nx*dx:dx]
x_coords = np.subtract(x_coords, x_center)
y_coords = np.subtract(y_coords, y_center)
radius = np.sqrt(np.multiply(x_coords, x_coords) + np.multiply(y_coords, y_coords))
   
fig = plt.figure()
plotter = PlotHelper.PlotHelper(fig, x_coords, y_coords, radius, \
                                eta0[validDomain[2]:validDomain[0], validDomain[3]:validDomain[1]],\
                                u0[validDomain[2]:validDomain[0], :], \
                                v0[:, validDomain[3]:validDomain[1]])
T = 50
def animate(i):
    if (i>0):
        #t = sim.step(1.0)
        t = sim.step(10.0)
    else:
        t = 0.0
    eta1, u1, v1 = sim.download()

    plotter.plot(eta1[validDomain[2]:validDomain[0], validDomain[3]:validDomain[1]],\
                 u1[validDomain[2]:validDomain[0], :], \
                 v1[:, validDomain[3]:validDomain[1]]);

    fig.suptitle("FBL Time = " + "{:04.0f}".format(t) + " s", fontsize=18)

    if (i%10 == 0):
        print "{:03.0f}".format(100.0*i / T) + " % => t=" + str(t)
        fig.savefig(imgdir + "/{:010.0f}_fbl.png".format(t))
            
anim = animation.FuncAnimation(fig, animate, range(T), interval=100)
plt.close(anim._fig)
anim

In [ ]:
#Centered in time, centered in space
nx = 100
ny = 200

dx = 200.0
dy = 200.0

dt = 1
g = 9.81
r = 0.0
A = 1
f = 0.01

#wind = Common.WindStressParams(type=0, tau0=3.0, rho=1025, alpha=1.0/(100*dx))
wind = Common.WindStressParams(type=99)

    

bcSettings = 1
ghosts = [1,1,1,1] # north, east, south, west
if (bcSettings == 1):
    boundaryConditions = Common.BoundaryConditions()
    # Wall boundary conditions
elif (bcSettings == 2):
    # periodic boundary conditions
    boundaryConditions = Common.BoundaryConditions(2,2,2,2)
elif bcSettings == 3:
    # periodic north-south
    boundaryConditions = Common.BoundaryConditions(2,1,2,1)
else:
    # periodic east-west
    boundaryConditions = Common.BoundaryConditions(1,2,1,2)

h0 = np.ones((ny+2, nx+2), dtype=np.float32, order='C') * 60;
#addTopographyBump(h0, nx, ny, dx, dy, ghosts, 40)

eta0 = np.zeros((ny+2, nx+2), dtype=np.float32, order='C');
u0 = np.zeros((ny+2, nx+1+2), dtype=np.float32, order='C');
v0 = np.zeros((ny+1+2, nx+2), dtype=np.float32, order='C');

#Create bump in to lower left of domain for testing
x_center = dx*nx*0.3
y_center = dy*ny*0.2
makeCentralBump(eta0, 0.0, nx, ny, dx, dy, ghosts)
#makeUpperCornerBump(eta0, nx, ny, dx, dy, ghosts)
#addDualVortexStaggered(eta0, u0, v0, nx, ny, dx, dy, ghosts)
    

#Initialize simulator
reload(CTCS)
sim = CTCS.CTCS(cl_ctx, \
                h0, eta0, u0, v0, \
                nx, ny, \
                dx, dy, dt, \
                g, f, r, A, \
                wind_stress=wind, \
                write_netcdf=True, \
                boundary_conditions=boundaryConditions )


#Calculate radius from center of bump for plotting
y_coords, x_coords = np.mgrid[0:ny*dy:dy, 0:nx*dx:dx]
x_coords = np.subtract(x_coords, x_center)
y_coords = np.subtract(y_coords, y_center)
radius = np.sqrt(np.multiply(x_coords, x_coords) + np.multiply(y_coords, y_coords))

ghosts = [-1,1,-1,1]

fig = plt.figure()
plotter = PlotHelper.PlotHelper(fig, x_coords, y_coords, radius, eta0[1:-1, 1:-1], u0[1:-1, :], v0[:, 1:-1])

T = 50
def animate(i):
    if (i>0):
        t = sim.step(10.0)
    else:
        t = 0.0
    eta1, u1, v1 = sim.download()

    plotter.plot(eta1[1:-1, 1:-1], u1[1:-1, :], v1[:, 1:-1]);
    fig.suptitle("CTCS Time = " + "{:04.0f}".format(t) + " s", fontsize=18)

    if (i%10 == 0):
        print "{:03.0f}".format(100.0*i / T) + " % => t=" + str(t)
        fig.savefig(imgdir + "/{:010.0f}_ctcs.png".format(t))

anim = animation.FuncAnimation(fig, animate, range(T), interval=100)
plt.close(anim._fig)
anim

In [ ]:
# Lax Friedrichs
reload(LxF)

nx = 100
ny = 200

dx = 20.0
dy = 20.0

dt = 1.5
g = 9.81

h0 = np.ones((ny+2, nx+2), dtype=np.float32, order='C') * 5;
u0 = np.zeros((ny+2, nx+2), dtype=np.float32, order='C');
v0 = np.zeros((ny+2, nx+2), dtype=np.float32, order='C');

#Create bump in center of domain for testing
addCentralBump(h0, nx, ny, dx, dy, ghosts)

            
#Initialize simulator
reload(LxF)
sim = LxF.LxF(cl_ctx, \
              h0, u0, v0, \
              nx, ny, \
              dx, dy, dt, \
              g)


#Calculate radius from center of bump for plotting
x_center = dx*nx/2.0
y_center = dy*ny/2.0
y_coords, x_coords = np.mgrid[0:ny*dy:dy, 0:nx*dx:dx]
x_coords = np.subtract(x_coords, x_center)
y_coords = np.subtract(y_coords, y_center)
radius = np.sqrt(np.multiply(x_coords, x_coords) + np.multiply(y_coords, y_coords))


fig = plt.figure()
plotter = PlotHelper.PlotHelper(fig, x_coords, y_coords, radius, h0[1:-1, 1:-1] - 5, u0[1:-1, 1:-1], v0[1:-1, 1:-1])

T = 50
def animate(i):
    if (i>0):
        t = sim.step(10.0)
    else:
        t = 0.0
    h1, u1, v1 = sim.download()

    plotter.plot(h1[1:-1, 1:-1] - 5.0, u1[1:-1, 1:-1], v1[1:-1, 1:-1]);
    fig.suptitle("LxF Time = " + "{:04.0f}".format(t) + " s", fontsize=18)

    if (i%10 == 0):
        print "{:03.0f}".format(100.0*i / T) + " % => t=" + str(t)
        fig.savefig(imgdir + "/{:010.0f}_lxf.png".format(t))
            
anim = animation.FuncAnimation(fig, animate, range(T), interval=100)
plt.close(anim._fig)
anim

In [ ]:
# Kurganov-Petrova 2007 paper
reload(KP07)
nx = 100
ny = 200

dx = 200.0
dy = 200.0

dt = 0.95/100
g = 9.81

f = 0.00
r = 0.0

# WIND
#wind = Common.WindStressParams(type=0, tau0=3.0, rho=1025, alpha=1.0/(100*dx))
wind = Common.WindStressParams(type=99)


bcSettings = 1
ghosts = np.array([2,2,2,2]) # north, east, south, west
validDomain = np.array([2,2,2,2])
if (bcSettings == 1):
    boundaryConditions = Common.BoundaryConditions()
elif (bcSettings == 2):
    boundaryConditions = Common.BoundaryConditions(2,2,2,2)
elif bcSettings == 3:
    # Periodic NS
    boundaryConditions = Common.BoundaryConditions(2,1,2,1)
else:
    # Periodic EW
    boundaryConditions = Common.BoundaryConditions(1,2,1,2)
    
dataShape = (ny + ghosts[0]+ghosts[2], 
             nx + ghosts[1]+ghosts[3])
waterHeight = 60
h0 = np.ones(dataShape, dtype=np.float32, order='C') * waterHeight;
u0 = np.zeros(dataShape, dtype=np.float32, order='C');
v0 = np.zeros(dataShape, dtype=np.float32, order='C');

Bi = np.zeros((dataShape[0]+1, dataShape[1]+1), dtype=np.float32, order='C')
#makeBathymetryCrater(Bi, nx+1, ny+1, dx, dy, ghosts)

#addCornerBump(h0, nx, ny, dx, dy, validDomain)
#addUpperCornerBump(h0, nx, ny, dx, dy, validDomain)
addCentralBump(h0, nx, ny, dx, dy, validDomain)
#addLowerLeftBump(h0, nx, ny, dx, dy, validDomain)
#addDebugBump(h0, nx, ny, dx, dy, 0.2, 0.1, validDomain)
#addDiagonalDam(h0, nx, ny, dx, dy, ghosts, 0.05)
#addStraightDam(h0, nx, ny, dx, dy, ghosts, 0.1)


#Initialize simulator
reload(KP07)
reload(Common)
sim = KP07.KP07(cl_ctx, \
                h0, Bi, u0, v0, \
                nx, ny, \
                dx, dy, dt, \
                g, f, r, \
                wind_stress=wind, \
                boundary_conditions=boundaryConditions,
                use_rk2=True)


#Calculate radius from center of bump for plotting
x_center = dx*nx/2.0
y_center = dy*ny/2.0
y_coords, x_coords = np.mgrid[0:ny*dy:dy, 0:nx*dx:dx]
x_coords = np.subtract(x_coords, x_center)
y_coords = np.subtract(y_coords, y_center)
radius = np.sqrt(np.multiply(x_coords, x_coords) + np.multiply(y_coords, y_coords))


fig = plt.figure()
plotter = PlotHelper.PlotHelper(fig, x_coords, y_coords, radius, 
                                h0[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]] - waterHeight, 
                                u0[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]], 
                                v0[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]] )

T = 50
def animate(i):
    if (i>0):
        t = sim.step(10.0)
    else:
        t = 0.0
    h1, u1, v1 = sim.download()

    plotter.plot(h1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]] - waterHeight, 
                 u1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]], 
                 v1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]] );
    fig.suptitle("KP07 Time = " + "{:04.0f}".format(t) + " s", fontsize=18)

    if (i%10 == 0):
        print "{:03.0f}".format(100*i / T) + " % => t=" + str(t) + "\tMax h: " + str(np.max(h1))
        #fig.savefig(imgdir + "/{:010.0f}_kp07.png".format(t))
             
anim = animation.FuncAnimation(fig, animate, range(T), interval=100)
plt.close(anim._fig)
anim

In [ ]:
# Kurganov-Petrova 2007 paper
reload(KP07)
nx = 100
ny = 200

dx = 200.0
dy = 200.0

dt = 0.95/100
g = 9.81

f = 0.00
r = 0.0

# WIND
#wind = Common.WindStressParams(type=0, tau0=3.0, rho=1025, alpha=1.0/(100*dx))
wind = Common.WindStressParams(type=99)


bcSettings = 1
ghosts = np.array([2,2,2,2]) # north, east, south, west
validDomain = np.array([2,2,2,2])
if (bcSettings == 1):
    boundaryConditions = Common.BoundaryConditions()
elif (bcSettings == 2):
    boundaryConditions = Common.BoundaryConditions(2,2,2,2)
elif bcSettings == 3:
    # Periodic NS
    boundaryConditions = Common.BoundaryConditions(2,1,2,1)
else:
    # Periodic EW
    boundaryConditions = Common.BoundaryConditions(1,2,1,2)
    
dataShape = (ny + ghosts[0]+ghosts[2], 
             nx + ghosts[1]+ghosts[3])
waterHeight = 60
h0 = np.ones(dataShape, dtype=np.float32, order='C') * waterHeight;
u0 = np.zeros(dataShape, dtype=np.float32, order='C');
v0 = np.zeros(dataShape, dtype=np.float32, order='C');

Bi = np.zeros((dataShape[0]+1, dataShape[1]+1), dtype=np.float32, order='C')
#makeBathymetryCrater(Bi, nx+1, ny+1, dx, dy, ghosts)

#addCornerBump(h0, nx, ny, dx, dy, validDomain)
#addUpperCornerBump(h0, nx, ny, dx, dy, validDomain)
addCentralBump(h0, nx, ny, dx, dy, validDomain)
#addLowerLeftBump(h0, nx, ny, dx, dy, validDomain)
#addDebugBump(h0, nx, ny, dx, dy, 0.2, 0.1, validDomain)
#addDiagonalDam(h0, nx, ny, dx, dy, ghosts, 0.05)
#addStraightDam(h0, nx, ny, dx, dy, ghosts, 0.1)


#Initialize simulator
reload(KP07)
reload(Common)
sim = KP07.KP07(cl_ctx, \
                h0, Bi, u0, v0, \
                nx, ny, \
                dx, dy, dt, \
                g, f, r, \
                wind_stress=wind, \
                boundary_conditions=boundaryConditions,
                use_rk2=False)


#Calculate radius from center of bump for plotting
x_center = dx*nx/2.0
y_center = dy*ny/2.0
y_coords, x_coords = np.mgrid[0:ny*dy:dy, 0:nx*dx:dx]
x_coords = np.subtract(x_coords, x_center)
y_coords = np.subtract(y_coords, y_center)
radius = np.sqrt(np.multiply(x_coords, x_coords) + np.multiply(y_coords, y_coords))


fig = plt.figure()
plotter = PlotHelper.PlotHelper(fig, x_coords, y_coords, radius, 
                                h0[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]] - waterHeight, 
                                u0[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]], 
                                v0[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]] )

T = 50
def animate(i):
    if (i>0):
        t = sim.step(10.0)
    else:
        t = 0.0
    h1, u1, v1 = sim.download()

    plotter.plot(h1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]] - waterHeight, 
                 u1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]], 
                 v1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]] );
    fig.suptitle("KP07 Time = " + "{:04.0f}".format(t) + " s", fontsize=18)

    if (i%10 == 0):
        print "{:03.0f}".format(100*i / T) + " % => t=" + str(t) + "\tMax h: " + str(np.max(h1))
        #fig.savefig(imgdir + "/{:010.0f}_kp07.png".format(t))
             
anim = animation.FuncAnimation(fig, animate, range(T), interval=100)
plt.close(anim._fig)
anim

In [ ]:
#Coriolis well balanced reconstruction scheme
nx = 100
ny = 200

dx = 200.0
dy = 200.0

dt = 0.95/10.0
g = 9.81

f = 0.00
r = 0.0

# WIND
#wind = Common.WindStressParams(type=0, tau0=3.0, rho=1025, alpha=1.0/(100*dx))
wind = Common.WindStressParams(type=99)

bcSettings = 1
ghosts = np.array([3,3,3,3]) # north, east, south, west
validDomain = np.array([3,3,3,3])
if (bcSettings == 1):
    boundaryConditions = Common.BoundaryConditions()
elif (bcSettings == 2):
    boundaryConditions = Common.BoundaryConditions(2,2,2,2)
elif bcSettings == 3:
    # Periodic NS
    boundaryConditions = Common.BoundaryConditions(2,1,2,1)
else:
    # Periodic EW
    boundaryConditions = Common.BoundaryConditions(1,2,1,2)
    
dataShape = (ny + ghosts[0]+ghosts[2], 
             nx + ghosts[1]+ghosts[3])


waterHeight = 60
h0 = np.ones(dataShape, dtype=np.float32, order='C') * waterHeight;
u0 = np.zeros(dataShape, dtype=np.float32, order='C');
v0 = np.zeros(dataShape, dtype=np.float32, order='C');

# Bathymetry:
Bi = np.zeros((dataShape[0]+1, dataShape[1]+1), dtype=np.float32, order='C')
#diagonalWallBathymetry(Bi, nx, ny, dx, dy, ghosts, 50)
#straightWallBathymetry(Bi, nx, ny, dx, dy, ghosts, 50)
#makeBathymetryCrater(Bi, nx+1, ny+1, dx, dy, ghosts)
#linearBathymetryX(Bi, nx, ny, dx, dy, ghosts, 5, 50)
#linearBathymetryY(Bi, nx, ny, dx, dy, ghosts, 5, 50)

#addCornerBump(h0, nx, ny, dx, dy, validDomain)
#addUpperCornerBump(h0, nx, ny, dx, dy, validDomain)
addCentralBump(h0, nx, ny, dx, dy, validDomain)
#addStraightDam(h0, nx, ny, dx, dy, ghosts, 0.1)



#Initialize simulator
reload(CDKLM16)
sim = CDKLM16.CDKLM16(cl_ctx, \
                h0, u0, v0, \
                Bi, \
                nx, ny, \
                dx, dy, dt, \
                g, f, r, \
                wind_stress=wind, \
                boundary_conditions=boundaryConditions, \
                reportGeostrophicEquilibrium=False)

#Calculate radius from center of bump for plotting
x_center = dx*nx/2.0
y_center = dy*ny/2.0
y_coords, x_coords = np.mgrid[0:ny*dy:dy, 0:nx*dx:dx]
x_coords = np.subtract(x_coords, x_center)
y_coords = np.subtract(y_coords, y_center)
radius = np.sqrt(np.multiply(x_coords, x_coords) + np.multiply(y_coords, y_coords))


fig = plt.figure()
plotter = PlotHelper.PlotHelper(fig, x_coords, y_coords, radius, 
                                h0[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]] - waterHeight, 
                                u0[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]], 
                                v0[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]])

T = 50
def animate(i):
    if (i>0):
        t = sim.step(10.0)

    else:
        t = 0.0
    h1, u1, v1 = sim.download()

    brighten = 1
    
    plotter.plot(h1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]] - waterHeight, 
                 brighten*u1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]], 
                 brighten*v1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]]);
    fig.suptitle("CDKLM16 Time = " + "{:04.0f}".format(t) + " s", fontsize=18)

    if (i%10 == 0):
        print "{:03.0f}".format(100*i / T) + " % => t=" + str(t) + "\tMax h: " + str(np.max(h1)) + \
        "\tMax u: " + str(np.max(u1)) + \
        "\tMax v: " + str(np.max(v1))
        
        fig.savefig(imgdir + "/{:010.0f}_cdklm16.png".format(t))
             
anim = animation.FuncAnimation(fig, animate, range(T), interval=100)
plt.close(anim._fig)
anim